In [ ]:
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import json
from pathlib import Path

from sem.generate_series import create_sde_process
from sem.sem.EM import NormalMixtureEM
from sem.sem.windows import Windows, calculate_acf

class ForecastingMixin:
    """
    Mixin class providing multi-step forecasting methods.
    Models should inherit from this mixin and nn.Module.
    """
    
    def stateless_forward_multistep(self, windows: torch.Tensor, n_steps: int) -> torch.Tensor:
        """
        Autoregressive multi-step forecasting for stateless models.
        
        Args:
            windows: [batch_size, window_size] - input windows
            n_steps: number of steps to forecast
            
        Returns:
            predictions: [batch_size, n_steps]
        """
        forecasts = []
        for _ in range(n_steps):
            forecast = self.forward(windows)
            forecasts.append(forecast)

            windows = torch.cat([
                windows[:, 1:],
                forecast.unsqueeze(-1)
            ], dim=1)
        results = torch.stack(forecasts, dim=1)
        return results
    
    @torch.no_grad()
    def generate_forecast(self, idx: int, 
                         time_series: torch.Tensor,
                         n_test_steps: int,
                         window_size: int) -> torch.Tensor:
        """
        Generate forecast starting at given index.
        
        Args:
            idx: starting index (negative values count from end)
            time_series: full time series data
            n_test_steps: number of steps to forecast
            window_size: size of input window
            train_series: alias for time_series (for backward compatibility)
            
        Returns:
            predictions: forecast values
        """
        if idx < 0:
            idx = time_series.shape[-1] + idx
        idx += 1

        start_idx = idx - window_size
        ts = time_series[..., start_idx: idx]
        assert ts.shape[-1] == window_size, 'not enough data'

        if not isinstance(ts, torch.Tensor):
            ts = torch.tensor(ts, dtype=torch.float32)
        if ts.dim() == 1:
            ts = ts.unsqueeze(0)

        device = next(self.parameters()).device
        window = ts.to(device)

        preds = self.forward_multistep(window, n_test_steps)
        if ts.shape[0] == 1:
            preds = preds.squeeze(0)
            
        return preds
    
    def get_name(self, add: str = "") -> str:
        """
        Get model name with optional suffix.
        
        Args:
            add: string to append to model name
            
        Returns:
            Model class name with suffix
        """
        class_name = self.__class__.__name__
        return class_name + add

    def forward_multistep(self, windows: torch.Tensor, n_steps: int) -> torch.Tensor:
        """
        Default multi-step forecasting implementation.
        Can be overridden by subclasses for stateful models.
        
        Args:
            windows: [batch_size, window_size]
            n_steps: number of steps to forecast
            
        Returns:
            predictions: [batch_size, n_steps]
        """
        return self.stateless_forward_multistep(windows, n_steps)


class EMForecaster(ForecastingMixin, nn.Module):
    def __init__(self, 
                 window_size: int,
                 dx_linear: bool,
                 n_components: int = 3,
                 hidden_dim: int = 64,
                 n_em_iters: int = 5):
        super().__init__()
        
        self.dwindow_size = window_size - 1
        self.window_size = window_size
        self.n_components = n_components
        negative_slope = 0.01
        self.dx_linear = dx_linear

        if dx_linear:
            self.dx_layer = nn.Linear(self.dwindow_size, self.dwindow_size, bias=False)

        self.em_layer = NormalMixtureEM(
            series_length=self.dwindow_size,
            n_components=n_components,
            n_sem_iters=n_em_iters
        )

        mlp_input_dim = self.dwindow_size * (n_components)
        
        self.mlp = nn.Sequential(
            nn.Linear(mlp_input_dim, 2 * hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(2 * hidden_dim, n_components, bias=False)
        )

        fusion_input_dim = n_components * 5
        self.fusion = nn.Sequential(
            nn.Linear(fusion_input_dim, hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(hidden_dim, 1, bias=False)
        )

        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_uniform_(module.weight, negative_slope)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

    def forward(self, X_window):
        batch_size = X_window.shape[0]

        dX1 = X_window[..., 1:] - X_window[..., :-1]
        if self.dx_linear:
            dX1 = self.dx_layer(dX1)
        g_ik, p_k, a_k, b_k = self.em_layer(dX1)

        dX_extended = (g_ik * dX1.unsqueeze(-1)).reshape((batch_size, -1))

        y_k = self.mlp(dX_extended)

        attention_logits = -b_k
        attention_weights = nn.functional.softmax(attention_logits, dim=1)

        fusion_input = torch.cat([
            y_k,
            attention_weights * y_k,
            a_k,
            attention_weights * a_k,
            p_k * a_k
        ], dim=1)
        forecast = X_window[..., -1] + self.fusion(fusion_input).squeeze(-1)
        
        return forecast


class MLP(ForecastingMixin, nn.Module):
    def __init__(self, window_size, hidden_dim=80):
        super().__init__()
        self.window_size = window_size
        self.dwindow_size = window_size - 1

        self.enrich_mlp = nn.Sequential(
            nn.Linear(self.dwindow_size, hidden_dim, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.01),
            nn.Linear(hidden_dim, hidden_dim, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.01),
            nn.Linear(hidden_dim, hidden_dim, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.01)
        )
        
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim + self.dwindow_size, hidden_dim // 2, bias=False),
            nn.LeakyReLU(0.01),
            nn.Linear(hidden_dim // 2, 1, bias=False)
        )
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, a=0.01)

    def forward(self, x):
        dx = x[:, 1:] - x[:, :-1]
        
        enriched = self.enrich_mlp(dx)
        
        combined = torch.cat([dx, enriched], dim=1)
        
        delta = self.fusion(combined).squeeze(-1)
        return x[:, -1] + delta


class TCNForecaster(ForecastingMixin, nn.Module):
    def __init__(self, window_size, hidden_dim=64, negative_slope=0.01):
        super().__init__()
        self.dwindow_size = window_size - 1
        
        self.dx_layer = nn.Linear(self.dwindow_size, self.dwindow_size, bias=False)

        self.tcn = nn.Sequential(
            nn.Conv1d(1, hidden_dim, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(negative_slope),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=2, dilation=2, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(negative_slope),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=4, dilation=4, bias=False),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(negative_slope),
            nn.AdaptiveAvgPool1d(1)
        )

        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim + self.dwindow_size, hidden_dim // 2, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(hidden_dim // 2, 1, bias=False)
        )
        
        self._init_params()
    
    def _init_params(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d, nn.Linear)):
                nn.init.kaiming_uniform_(m.weight, a=0.01, mode='fan_in')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, X_window):
        dX = X_window[:, 1:] - X_window[:, :-1]
        dX_reshaped = dX.unsqueeze(1)
        tcn_features = self.tcn(dX_reshaped).squeeze(-1)

        combined = torch.cat([dX, tcn_features], dim=1)
        
        delta = self.fusion(combined).squeeze(-1)
        return X_window[:, -1] + delta


class LightGRU(ForecastingMixin, nn.Module):
    def __init__(self, window_size, hidden_size=64, proj_size=32):
        super().__init__()
        self.window_size = window_size
        self.dwindow_size = window_size - 1
        
        self.input_proj = nn.Conv1d(1, proj_size, 1, bias=False)
        
        self.gru = nn.GRU(
            input_size=proj_size,
            hidden_size=hidden_size,
            num_layers=2,
            batch_first=True,
            bias=False
        )
        
        self.fusion = nn.Sequential(
            nn.Linear(hidden_size + self.dwindow_size, hidden_size // 2, bias=False),
            nn.LeakyReLU(0.01),
            nn.Linear(hidden_size // 2, 1, bias=False)
        )

        self._init_weights()
    
    def _init_weights(self):
        for name, param in self.gru.named_parameters():
            if 'weight' in name:
                nn.init.orthogonal_(param)
        
        for m in self.modules():
            if isinstance(m, (nn.Linear, nn.Conv1d)):
                nn.init.kaiming_uniform_(m.weight, a=0.01, mode='fan_in')
    
    def forward(self, x, hidden=None):

        dx = x[:, 1:] - x[:, :-1]
        dx_reshaped = dx.unsqueeze(1)
        dx_proj = self.input_proj(dx_reshaped)
        dx_proj = dx_proj.transpose(1, 2)
        output, hidden = self.gru(dx_proj, hidden)
        
        gru_features = output[:, -1, :]
        combined = torch.cat([dx, gru_features], dim=1)
        delta = self.fusion(combined).squeeze(-1)
        prediction = x[:, -1] + delta
        
        return prediction, hidden
    
    def forward_multistep(self, windows, n_steps):
        predictions = []
        current_window = windows.clone()
        hidden = None
        
        for _ in range(n_steps):
            pred, hidden = self.forward(current_window, hidden)
            predictions.append(pred.unsqueeze(1))
            current_window = torch.cat([current_window[:, 1:], pred.unsqueeze(1)], dim=1)
        
        return torch.cat(predictions, dim=1)


class LightAttention(ForecastingMixin, nn.Module):
    def __init__(self, window_size, embed_dim=64, num_heads=8, num_layers=2):
        super().__init__()
        self.window_size = window_size
        self.dwindow_size = window_size - 1
        
        self.embed = nn.Conv1d(1, embed_dim, 1, bias=False)
        
        self.position_enc = nn.Parameter(torch.randn(1, self.dwindow_size, embed_dim))
        
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(
                embed_dim, num_heads,
                dropout=0.1,
                bias=False,
                batch_first=True
            )
            for _ in range(num_layers)
        ])
        
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(embed_dim)
            for _ in range(num_layers)
        ])
        
        self.fusion = nn.Sequential(
            nn.Linear(embed_dim + self.dwindow_size, embed_dim, bias=False),
            nn.LayerNorm(embed_dim),
            nn.LeakyReLU(0.01),
            nn.Linear(embed_dim, embed_dim // 2, bias=False),
            nn.LayerNorm(embed_dim // 2),
            nn.LeakyReLU(0.01),
            nn.Linear(embed_dim // 2, 1, bias=False)
        )
        
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Linear, nn.Conv1d)):
                nn.init.kaiming_uniform_(m.weight, a=0.01, mode='fan_in')
            elif isinstance(m, nn.LayerNorm):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        dx = x[:, 1:] - x[:, :-1]
        
        dx_reshaped = dx.unsqueeze(1)
        dx_embedded = self.embed(dx_reshaped)
        dx_embedded = dx_embedded.transpose(1, 2)
        dx_embedded = dx_embedded + self.position_enc
        
        attn_out = dx_embedded
        for attn_layer, norm in zip(self.attention_layers, self.layer_norms):
            residual = attn_out
            attn_out, _ = attn_layer(attn_out, attn_out, attn_out)
            attn_out = norm(residual + attn_out)
        
        attn_features = attn_out[:, -1, :]
        
        combined = torch.cat([dx, attn_features], dim=1)
        delta = self.fusion(combined).squeeze(-1)
        
        return x[:, -1] + delta

In [ ]:
from time import time
import json

class TimeSeriesDataset(Dataset):
    def __init__(self, time_series: np.ndarray, window_size, n_train_steps=1):
        super().__init__()
        self.ts = time_series
        self.n_train_steps = n_train_steps
        self.window_length = window_size

    def __len__(self):
        return len(self.ts) - self.window_length - self.n_train_steps

    def __getitem__(self, idx):
        idx_last = idx + self.window_length
        return self.ts[idx:idx_last], self.ts[idx_last: idx_last + self.n_train_steps]


class MAPELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps
    
    def forward(self, predictions, targets):
        mape = torch.abs(predictions - targets) / (torch.abs(targets) + self.eps)
        return mape.mean()


class WeightedLoss(nn.Module):
    def __init__(self, mape=0.1, mse=0.9, eps=1e-8):
        super().__init__()
        self.mape = MAPELoss(eps)
        self.mse = nn.MSELoss()
        self.mape_weight = mape
        self.mse_weight = mse

    def forward(self, predictions, targets):
        return self.mape_weight * self.mape(predictions, targets) + self.mse_weight * self.mse(predictions, targets)


class Trainer:
    def __init__(self, setup, time_series, window_size, len_train, log_path, batch_size=64, 
                 log_freq_batch=50, n_tests=100, n_test_steps=50, n_train_steps=1, eps=1e-8, device='cuda'):
        self.train_series = time_series[:len_train]
        self.eval_series = time_series[len_train:]
        dataset = TimeSeriesDataset(self.train_series, window_size=window_size, n_train_steps=n_train_steps)
        self.dataloader = DataLoader(dataset, batch_size, shuffle=True)

        self.window_size = window_size
        self.n_epochs = setup['n_epochs']
        self.n_tests = n_tests
        self.n_test_steps = n_test_steps
        self.model = setup['model_class'](**setup['model_args']).to(device)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), setup['lr'])
        self.criterion = WeightedLoss()
        self.scheduler = torch.optim.lr_scheduler.LinearLR(
            self.optimizer, 1.0, 1e-8, self.n_epochs
        )

        self.loss_threshold = 0.01
        self.eps = eps
        self.device = device
        self.log_freq_batch = log_freq_batch
        self.log_path = log_path
        
        self.n_train_steps = n_train_steps

    @torch.no_grad()
    def validate(self):
        preds = self.model.generate_forecast(
            idx=-1,
            time_series=self.train_series,
            n_test_steps=self.n_test_steps,
            window_size=self.window_size
        ).cpu().numpy()
        target = self.eval_series[:self.n_test_steps]
        return preds, target
    
    @torch.no_grad()
    def evaluate_multistep(self, test_ids):
        preds = np.empty((self.n_tests + 1, self.n_test_steps))
        targets = np.empty((self.n_tests + 1, self.n_test_steps))
        for test in range(self.n_tests):
            idx = test_ids[test]
            target_idx = idx + 1
            pred = self.model.generate_forecast(
                idx=idx,
                time_series=self.train_series,
                n_test_steps=self.n_test_steps,
                window_size=self.window_size
            ).cpu()
            target = self.train_series[target_idx:target_idx + self.n_test_steps]
            preds[test, :] = pred.numpy()
            targets[test, :] = target
        return preds, targets

    def run(self, test_ids):
        epoch_train_time = 0
        epoch_val_time = 0
        start_full = time()
        for epoch in range(1, self.n_epochs + 1):
            self.model.train()
            start_train = time()
            avg_threshold = self.train_epoch(epoch)
            epoch_train_time += time() - start_train
            self.model.eval()

            start_test = time()
            preds, targets = self.evaluate_multistep(test_ids)
            pred, target = self.validate()
            epoch_val_time += time() - start_test
            preds[-1, :] = pred
            targets[-1, :] = target
            tests_results = np.stack([preds, targets], axis=-1)  # shape: [n_tests+1, n_steps, 2]
            
            np.save(self.log_path / f'epoch_{epoch}', tests_results)
            
            if avg_threshold < self.loss_threshold:
                break
        
        time_full = time() - start_full
        mean_train_time = epoch_train_time / epoch
        mean_test_time = epoch_val_time / epoch / (self.n_test_steps + 1) / (self.n_tests)

        json.dump(
                {
                    'full_time': float(time_full),
                    'epoch_train_time': float(mean_train_time),
                    'epoch_test_time_one_step': float(mean_test_time),
                    'last_loss': float(avg_threshold)
                },
                open(self.log_path / 'times.json', 'w'),
                indent=2
            )
        print()
        print("Training completed!")
        return self.model
    
    def train_epoch(self, epoch_idx=0):
        print("Epoch:", epoch_idx)
        total_loss = 0.0

        for batch_idx, (windows, y) in enumerate(self.dataloader):
            windows = windows.to(self.device).float()
            y = y.to(self.device).float()
            if self.n_train_steps == 1:
                y = y.squeeze(-1)

            self.optimizer.zero_grad()

            if self.n_train_steps == 1:
                results = self.model.forward(windows)
                if isinstance(results, tuple):
                    results = results[0]
            else:
                results = self.model.forward_multistep(windows, self.n_train_steps)

            loss = self.criterion(results, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
            self.optimizer.step()

            total_loss += loss.item()

            if batch_idx % self.log_freq_batch == 0:
                basic_metrics = self.compute_basic_metrics(results, y)
                print("Batch IDX:", batch_idx)
                print(basic_metrics)
                print()
            if torch.isnan(loss):
                print(f"WARNING: NaN loss at batch {batch_idx}")
                break
        self.scheduler.step()

        avg_loss = total_loss / len(self.dataloader)
        print(f"Epoch {epoch_idx} - Average Loss: {avg_loss:.6f}")
        return avg_loss

    @torch.no_grad()
    def compute_basic_metrics(self, forecast, y_true):
        mae = torch.abs(forecast - y_true).mean().item()
        mse = torch.mean((forecast - y_true) ** 2).item()
        mape = torch.abs((forecast - y_true) / (y_true.abs() + self.eps)).mean().item() * 100
        return {'MAE': mae, 'MSE': mse, 'MAPE': mape}


In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_setups(window_size):
    base_configs = []
    
    # MLP Model
    base_configs.append({
        'model_class': MLP,
        'model_args': {'window_size': window_size, 'hidden_dim': 80},
        'name': f'MLP',
        'lr': 1e-3,
        'n_epochs': 150
    })
    
    # TCN Model
    base_configs.append({
        'model_class': TCNForecaster,
        'model_args': {'window_size': window_size, 'hidden_dim': 64},
        'name': f'TCN',
        'lr': 1e-3,
        'n_epochs': 150
    })
    
    # LightAttention Model
    base_configs.append({
        'model_class': LightAttention,
        'model_args': {'window_size': window_size, 'embed_dim': 64, 'num_heads': 4},
        'name': 'LightAttention',
        'lr': 1e-3,
        'n_epochs': 150,
    })
    
    # LightGRU Model
    base_configs.append({
        'model_class': LightGRU,
        'model_args': {'window_size': window_size, 'hidden_size': 64, 'proj_size': 64},
        'name': 'LightGRU',
        'lr': 1e-2,
        'n_epochs': 300
    })
    
    # EMForecaster Model
    em_configs = [
        {'n_components': 2, 'dx_linear': False},
        {'n_components': 3, 'dx_linear': False},
        {'n_components': 2, 'dx_linear': True},
        {'n_components': 3, 'dx_linear': True},
    ]
    
    for config in em_configs:
        base_configs.append({
            'model_class': EMForecaster,
            'model_args': {
                'window_size': window_size,
                'dx_linear': config['dx_linear'],
                'n_components': config['n_components'],
                'hidden_dim': 64,
                'n_em_iters': 5
            },
            'name': f'EM_n{config["n_components"]}_dx_{config["dx_linear"]}',
            'lr': 1e-3,
            'n_epochs': 150
        })
    
    return base_configs

def create_experiment_folder(base_path="experiments", model_name=None, series_idx=None):
    """
    Create folder structure: experiments/<model_name>/<series_idx>/
    """
    if model_name is None:
        model_name = "default"
    if series_idx is None:
        series_idx = "0"
    
    exp_path = Path(base_path) / model_name / str(series_idx)
    exp_path.mkdir(parents=True, exist_ok=True)
    
    return exp_path

def generate_test_ids(window_size, train_series, n_tests, n_test_steps):
    res = []
    for _ in range(n_tests):
        res.append(random.randint(window_size, len(train_series) - 1 - n_test_steps))
    return res

In [4]:
from torchinfo import summary

window_size = 100

for setup in get_setups(window_size):
    model = setup['model_class'](**setup['model_args'])
    print(setup['name'])
    print(summary(model, input_data=torch.ones((1, window_size))))
    print()

Baseline
Layer (type:depth-idx)                   Output Shape              Param #
Baseline                                 [1]                       --
├─Sequential: 1-1                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 32]                   3,168
│    └─LeakyReLU: 2-2                    [1, 32]                   --
│    └─Linear: 2-3                       [1, 1]                    32
Total params: 3,200
Trainable params: 3,200
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01

EM_n2
Layer (type:depth-idx)                   Output Shape              Param #
EMForecaster                             [1]                       --
├─NormalMixtureEM: 1-1                   [1, 99, 2]                107
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 32]             

In [ ]:
n_tests = 10
series_length = 5000
train_length = 4500
alpha = 0.75
N_init = 5
N_add = 5
n_epoch_tests = 40
n_test_steps = 200

for test in range(n_tests):
    print('_' * 50)
    print(test)
    print('_' * 50)
    set_seed(10 * test)
    series = create_sde_process(series_length)['X']
    train_series = series[:train_length]
    *_, window_size = Windows()(train_series[1:] - train_series[:-1], alpha, N_init, N_add)
    window_size += 1
    setups = get_setups(window_size)
    test_ids = generate_test_ids(window_size, train_series, n_epoch_tests, n_test_steps)
    for setup in setups:
        set_seed(10 * test)
        print('_' * 50)
        print(setup['name'])
        print('_' * 50)

        log_path = create_experiment_folder(model_name=setup['name'], series_idx=test)
        if len(os.listdir(log_path)):
            print(log_path)
            continue
        trainer = Trainer(setup, series, window_size, train_length, log_path,
                          batch_size=64, log_freq_batch=70, n_tests=n_epoch_tests, n_test_steps=n_test_steps, device='cuda')
        model = trainer.run(test_ids)

__________________________________________________
0
__________________________________________________
N = 5; Max ACF(1): 0.9999998807907104
N = 10; Max ACF(1): 0.971988320350647
N = 15; Max ACF(1): 0.896878719329834
N = 20; Max ACF(1): 0.861565351486206
N = 25; Max ACF(1): 0.8270503878593445
N = 30; Max ACF(1): 0.8007834553718567
N = 35; Max ACF(1): 0.793499767780304
N = 40; Max ACF(1): 0.7914840579032898
N = 45; Max ACF(1): 0.789178729057312
N = 50; Max ACF(1): 0.7675512433052063
N = 55; Max ACF(1): 0.7647435069084167
N = 60; Max ACF(1): 0.7662709355354309
N = 65; Max ACF(1): 0.7611182332038879
N = 70; Max ACF(1): 0.7609160542488098
N = 75; Max ACF(1): 0.7589223384857178
N = 80; Max ACF(1): 0.7473089098930359
N = 85; Max ACF(1): 0.7415372729301453
N = 90; Max ACF(1): 0.7400791049003601
N = 95; Max ACF(1): 0.7308598160743713
N = 100; Max ACF(1): 0.7267736196517944
N = 105; Max ACF(1): 0.7119746804237366
Found window length: 105
__________________________________________________
Basel

KeyboardInterrupt: 